In [18]:
import pandas as pd
import re


In [ ]:
# 前提0：构建好了年份检测函数，并位于同1个目录下：车型年份检索.py
from 车型年份检索 import *

In [20]:
# 前提1：导入需要分类的ASIN和文本
df = pd.read_excel(io="需要车型分类文本.xlsx",sheet_name="Sheet1").loc[:,["asin","文本"]].drop_duplicates()
# 前提1.1 文本需要全部转化为小写
df["文本"] = df["文本"].astype(str).str.lower()
print(df.head())

         asin                                                 文本
0  B0DJX5X6LD  all weather floor mats for chevy silverado 200...
4  B07V1V29RC  yitamotor floor mats compatible with toyota tu...
5  B07M8SXG6B  oedro floor mats fit for honda accord sedan 20...
6  B0D7HXWQNY  karpal floor mats fit for 2015-2020 chevrolet ...
7  B08K3D28PC  lasfit front floor mats for ford f-150 2015-20...


In [21]:
# 前提2：导入已经建立好的车型数据库
车型数据 = pd.read_excel(io="C:/Users/Administrator/Desktop/车型数据.xlsx",sheet_name="Sheet1").drop_duplicates()
# 前提2.1 品牌和型号需要全部转化为小写
车型数据["品牌"] = 车型数据["品牌"].astype(str).str.lower()
车型数据["型号"] = 车型数据["型号"].astype(str).str.lower()
print(车型数据.head())

              品牌       型号
0         toyota  corolla
1  mercedes benz      gle
2            kia       k5
3            kia      rio
4            kia     niro


In [22]:
# 步骤1 匹配车辆的型号
# 步骤1.1 构建目标字符串检测函数
def find_string_occurrences(text, target, case_sensitive=False, whole_word=False):
    """
    检测目标字符串是否出现在文本中，并返回所有匹配位置和内容
    
    参数:
        text (str): 待检测的文本
        target (str): 要查找的目标字符串
        case_sensitive (bool): 是否区分大小写，默认不区分
        whole_word (bool): 在文本中查找目标字符串时，是否要求目标字符串作为一个独立的完整单词出现，而不是作为其他单词的一部分出现，默认否
    
    返回:
        dict: 包含匹配结果的字典，包括:
            - exists: 是否存在匹配
            - count: 匹配次数
            - positions: 每个匹配的起始和结束索引
            - occurrences: 每个匹配的具体内容
    """
    # 构建正则表达式模式
    pattern = re.escape(target)  # 转义特殊字符
    
    # 如果需要匹配整个单词，添加单词边界
    if whole_word:
        pattern = r'\b' + pattern + r'\b'
    
    # 设置正则表达式标志
    flags = 0
    if not case_sensitive:
        flags = re.IGNORECASE
    
    # 查找所有匹配
    matches = re.finditer(pattern, text, flags=flags)
    
    # 提取匹配信息
    result = {
        'exists': False,
        'count': 0,
        'positions': [],
        'occurrences': []
    }
    
    for match in matches:
        result['exists'] = True
        result['count'] += 1
        result['positions'].append((match.start(), match.end()))
        result['occurrences'].append(match.group())
    
    return result

# 示例用法
if __name__ == "__main__":
    test_text = """Python是一种流行的编程语言。
    python以其简洁的语法而闻名，很多人喜欢Python。
    学习Python编程可以打开很多机会，Python3是最新版本。"""
    
    # 测试1：基本匹配（不区分大小写）
    print("测试1：查找 'Python'（不区分大小写）")
    result = find_string_occurrences(test_text, "Python")
    print(f"是否存在: {result['exists']}")
    print(f"出现次数: {result['count']}")
    print(f"出现位置: {result['positions']}\n")

测试1：查找 'Python'（不区分大小写）
是否存在: True
出现次数: 5
出现位置: [(0, 6), (22, 28), (44, 50), (58, 64), (75, 81)]



In [23]:
# 步骤1.2，检测每个ASIN的标题中出现了哪些车辆型号

# 假设这是你型号列表，包含多选型号
# 车型数据['型号'] = ['chevy|chevrolet', 'chevy|chevrolet|bbq', 'ford', 'toyota']

# 获取唯一型号列表
unique_brands = 车型数据['型号'].unique()

# 为每型号创建一列
for brand in unique_brands:
    # 分型号的多个可选名称 
    brand_variants = brand.split('|')
    
    # 检查标题中是否包含任一可选名称
    def check_brand(title):
        for variant in brand_variants:
            if find_string_occurrences(title, variant)['exists']:
                return 1
        return 0
    
    # 应用检查函数到所有标题
    df[brand] = df['文本'].apply(check_brand)

print(df.head())

         asin                                                 文本  corolla  \
0  B0DJX5X6LD  all weather floor mats for chevy silverado 200...        0   
4  B07V1V29RC  yitamotor floor mats compatible with toyota tu...        0   
5  B07M8SXG6B  oedro floor mats fit for honda accord sedan 20...        0   
6  B0D7HXWQNY  karpal floor mats fit for 2015-2020 chevrolet ...        0   
7  B08K3D28PC  lasfit front floor mats for ford f-150 2015-20...        0   

   gle  k5  rio  niro  challenger  hrv  titan  ...  crx  crz  crosstour  \
0    0   0    0     0           0    0      0  ...    0    0          0   
4    0   0    0     0           0    0      0  ...    0    0          0   
5    0   0    0     0           0    0      0  ...    0    0          0   
6    0   0    0     0           0    0      0  ...    0    0          0   
7    0   0    0     0           0    0      0  ...    0    0          0   

   gladiator  journey  dakota  econoline super duty  e-450 super duty  e-250  \
0     

C:\Users\Administrator\AppData\Local\Temp\ipykernel_31376\1999031068.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[brand] = df['文本'].apply(check_brand)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31376\1999031068.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[brand] = df['文本'].apply(check_brand)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31376\1999031068.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor p

In [24]:
# 步骤1.3 将关系矩阵降维并通过增加行的方式实现文本与每个匹型号的一一对应
'''
示例效果：
如果原始数据是：
文本    F150    ram 1500    ram 2500
文本1   1   1   0
文本2   0   0   1
处理后会变为：
文本    型号
文本1	 f150
文本1   ram 1500
文本2   ram 2500
'''
# 假设df中已包含原始文本列和型号的0/1标识列
# 先获取所型号列名（排除原始文本列），从最长的型号开始匹配（越长的型号代表分类越细微，误判概率较小，越短的型号先匹配，则误判的概率更大）
# 假设车型数据['型号']中包含多选项型号
unique_brands = 车型数据['型号'].unique()

# 按长度排序型号（长型号优先）
brand_with_length = [(brand, len(brand)) for brand in unique_brands]
sorted_brands = [brand for brand, _ in sorted(brand_with_length, key=lambda x: -x[1])]

# 为每个型号创建标识列
for brand in sorted_brands:
    brand_variants = brand.split('|')
    brand_variants_sorted = sorted(brand_variants, key=lambda x: -len(x))
    pattern = r'\b(' + '|'.join(re.escape(v) for v in brand_variants_sorted) + r')\b'
    df[brand] = df['文本'].str.contains(pattern, case=False).astype(int)

# 识别所有型号列
brand_columns = [col for col in df.columns if col not in ['asin', '文本']]

# 新增一列检查是否有任何型号匹配
df['有匹配型号'] = df[brand_columns].any(axis=1).astype(int)

# 转换为长表
melted_df = df.melt(
    id_vars=['asin', '文本', '有匹配型号'],
    value_vars=brand_columns,
    var_name='型号',
    value_name='是否包含'
)

# 筛选出有匹配的记录
matched_df = melted_df[melted_df['是否包含'] == 1].drop(columns=['是否包含', '有匹配型号'])

# 处理无匹配的记录
no_match_df = df[df['有匹配型号'] == 0][['asin', '文本']].copy()
no_match_df['型号'] = '无匹配车型'

# 合并两种情况的结果
result_df = pd.concat([matched_df, no_match_df], ignore_index=True)

print(result_df)
    
    

C:\Users\Administrator\AppData\Local\Temp\ipykernel_31376\615169222.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[brand] = df['文本'].str.contains(pattern, case=False).astype(int)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31376\615169222.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[brand] = df['文本'].str.contains(pattern, case=False).astype(int)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31376\615169222.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[brand] = df['文本'].str.contains(pattern, case=False).astype(int)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31376\615169222.py:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To act

           asin                                                 文本       型号
0    B0DK6W8SFJ  ginowy-floor mats for toyota corolla 2009-2013...  corolla
1    B0CYM2YNBQ  kelcseecs floor mats custom for 2009 2010 2011...  corolla
2    B0DK2SCDFR  floor mats custom for toyota corolla 2009-2013...  corolla
3    B0D2W3VMFS  all weather floor mat fit for toyota corolla(i...  corolla
4    B0DPHJMJ9Q  g-plus floor mats 3pcs compatible with toyota ...  corolla
..          ...                                                ...      ...
752  B005ZWE0NG                           toyota carpet floor mats    无匹配车型
753  B0CX75LZ2W  husky liners weatherbeater floor mats | fits 2...    无匹配车型
754  B09QMJ9N18  mixsuper front floor mats compatible with 2015...    无匹配车型
755  B0D4YKL4L5  karpal floor mats for honda hr-v 2026 2025 202...    无匹配车型
756  B07K3SLLCZ  husky liners weatherbeater floor mats | fits 2...    无匹配车型

[757 rows x 3 columns]


In [25]:
# 步骤2 通过已经匹配好的型号字段，匹配对应的车辆品牌
# 直接合并并添加品牌列
result_df = result_df.merge(
    车型数据[['型号', '品牌']], 
    on='型号', 
    how='left'  # 保留result_df所有记录，没有匹配的品牌为NaN
)
print(result_df)

           asin                                                 文本       型号  \
0    B0DK6W8SFJ  ginowy-floor mats for toyota corolla 2009-2013...  corolla   
1    B0CYM2YNBQ  kelcseecs floor mats custom for 2009 2010 2011...  corolla   
2    B0DK2SCDFR  floor mats custom for toyota corolla 2009-2013...  corolla   
3    B0D2W3VMFS  all weather floor mat fit for toyota corolla(i...  corolla   
4    B0DPHJMJ9Q  g-plus floor mats 3pcs compatible with toyota ...  corolla   
..          ...                                                ...      ...   
752  B005ZWE0NG                           toyota carpet floor mats    无匹配车型   
753  B0CX75LZ2W  husky liners weatherbeater floor mats | fits 2...    无匹配车型   
754  B09QMJ9N18  mixsuper front floor mats compatible with 2015...    无匹配车型   
755  B0D4YKL4L5  karpal floor mats for honda hr-v 2026 2025 202...    无匹配车型   
756  B07K3SLLCZ  husky liners weatherbeater floor mats | fits 2...    无匹配车型   

         品牌  
0    toyota  
1    toyota  
2    toyo

In [ ]:
# 步骤3 在标题中匹配出对应品牌+车型的所属年份
# 步骤3.1 利用车型年份检索，筛选出文本中的年份范围
标注数据路径 = "C:/Users/Administrator/Desktop/车型标注数据.xlsx"  # 按之前的格式准备
位置规律 = 统计位置规律(标注数据路径)

temp = []
for text, model in zip(result_df['文本'], result_df['型号']):
    # 拆分型号（处理包含|的情况）
    model_variants = model.split('|')
    year = None
    
    # 逐个尝试型号变体
    for variant in model_variants:
        year = 提取指定型号的年份(text, variant.strip(), 位置规律)
        if year is not None:  # 如果找到年份则停止尝试
            break
    
    temp.append(year)

result_df['年份范围'] = temp
    

未找到canyon在1980-2100之间的年份
未找到yukon在1980-2100之间的年份
未找到tahoe在1980-2100之间的年份
未找到tahoe在1980-2100之间的年份
未找到accord在1980-2100之间的年份
未找到accord在1980-2100之间的年份
未找到ranger在1980-2100之间的年份
未找到ranger在1980-2100之间的年份
未找到x3在1980-2100之间的年份
未找到x7在1980-2100之间的年份
未找到x5在1980-2100之间的年份
未找到rogue在1980-2100之间的年份
未找到f-150在1980-2100之间的年份
未找到f150在1980-2100之间的年份
未找到f-150在1980-2100之间的年份
未找到f150在1980-2100之间的年份
未找到型号：f-150
未找到型号：f150
未找到f-150在1980-2100之间的年份
未找到f150在1980-2100之间的年份
未找到型号：f-150
未找到型号：f150
未找到型号：f-150
未找到型号：f150
未找到型号：f-150
未找到型号：f150
未找到型号：f-150
未找到型号：f150
未找到型号：f-150
未找到型号：f150
未找到f-150在1980-2100之间的年份
未找到f150在1980-2100之间的年份
未找到型号：f-150
未找到型号：f150
未找到型号：f-150
未找到型号：f150
未找到型号：cx-5
未找到型号：cx-5
未找到ram 1500在1980-2100之间的年份
未找到ram 1500在1980-2100之间的年份
未找到ram 1500在1980-2100之间的年份
未找到ram 1500在1980-2100之间的年份
未找到silverado在1980-2100之间的年份
未找到型号：cr-v
未找到cr-v在1980-2100之间的年份
未找到型号：cr-v
未找到型号：cr-v
未找到型号：cr-v
未找到型号：cr-v
未找到equinox在1980-2100之间的年份
未找到civic在1980-2100之间的年份
未找到trax在1980-2100之间的年份
未找到bronco在1980-2100之间的年份
未找到crosstr

In [ ]:
# 步骤3.2 将同个车型中，交集年份有2个的记录合并，如2014-2021与2017-2023合并为2014-2023，而2014-2021与